# Connect to datalake in Azure Storage Account

Run this code to mount your Azure Storage Account to Databricks.

This assumes you have a storage account `dbcopgeospatial`. Remember to remove the storage account key before committing.

In [0]:
import os

# Credentials
STORAGE_ACCOUNT_NAME = "dbcopgeospatial"
STORAGE_ACCOUNT_KEY = ""

# Paths
PATH_EXTERNAL_BLOB = f"wasbs://external@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/"
PATH_DATALAKE = f"wasbs://copgeospatial@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/"
PATH_ANALYSIS = f"wasbs://analysis@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/"

# Connect blob
spark.conf.set(
    f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.blob.core.windows.net",
    STORAGE_ACCOUNT_KEY,
)

# Mounts
def mount(mount_name: str, source: str, mount_root: str = "/mnt") -> None:
    """
    Mounts source location to local path under mount_local_root. Will ignore if mount
    alreay exits.

    Note: This function expects access to global STORAGE_ACCOUNT_NAME and
    STORAGE_ACCOUNT_KEY constants.

    Parameters:
        - local_name: mounted local directory name (subdirectory of mount_local_root)
        - source: location of source (e.g. blob container)
        - mount_local_root (optional): the root path of the local mounts
    """
    if f"{mount_name}/" not in [
        fileinfo.name for fileinfo in dbutils.fs.ls(mount_root)
    ]:
        print(mount_name)
        dbutils.fs.mount(
            source=source,
            mount_point=os.path.join(mount_root, mount_name),
            extra_configs={
                f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.blob.core.windows.net": STORAGE_ACCOUNT_KEY
            },
        )


for name, container in zip(
    ["copgeospatial"],
    [PATH_DATALAKE],
):
    mount(name, container)